In [2]:
from pathlib import Path
import re
from typing import Any
from pydantic import Field, BaseModel

from grasp_agents import (
    BaseTool,
    LLMAgent,
    LLMPromptArgs,
    RunContext,
    RunArgs,
    Packet,
    ImageData,
    Messages,
)
from grasp_agents.openai import OpenAILLM, OpenAILLMSettings
from grasp_agents.grasp_logging import setup_logging
from grasp_agents.packet_pool import PacketPool
from grasp_agents.comm_processor import DynCommPayload
from grasp_agents.utils import get_timestamp
from grasp_agents.workflow.sequential_workflow import SequentialWorkflow

Set up logging to write to the console and/or file

In [2]:
PACKAGE_DIR = Path.cwd()
LOGGING_DIR = Path.cwd() / "data/multiagent/logs"

In [3]:
LOGGING_CFG_PATH = PACKAGE_DIR / "configs/logging/default.yaml"
setup_logging(
    LOGGING_DIR / f"grasp_agents_demo_{get_timestamp()}.log", LOGGING_CFG_PATH
)

Paths to images used in the demo

In [4]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = PACKAGE_DIR / "src/grasp_agents/examples/data/expr.jpeg"

Utils

In [5]:
def print_single_output(out: Any) -> None:
    print(f"\n<Response>\n{out.payloads[0]}")

## Simple generation with validated outputs

Output type validation

In [6]:
# list[int] is the output type used to validate the output
chatbot = LLMAgent[None, list[int], None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="openai:gpt-4.1", llm_settings=OpenAILLMSettings(logprobs=True)
    ),
)

# This initialises printer and usage tracker
ctx = RunContext[None](print_messages=True)

In [7]:
# Code block delimiters are stripped from the output
out = await chatbot.run(
    "Output a list of 3 integers from 0 to 10 as a python array, no talking",
    ctx=ctx,
)
print_single_output(out)


<chatbot>[USER]
Output a list of 3 integers from 0 to 10 as a python array, no talking

<chatbot>[ASSISTANT]
[
  3,
  7,
  1
]

------------------------------------
I/O/(R)/(C) tokens: 27/9/0/0

<Response>
[3, 7, 1]


Completion data (e.g. log probs) per agent can be accessed via RunContext:

In [8]:
# ctx.completions

Output type validation with structured outputs

In [9]:
# Some providers (e.g. `openai` and `google_ai_studio`) support structured outputs.
# With the OpenAI API, this will require a Pydantic model to validate the output.

from enum import StrEnum


class Selector(StrEnum):
    A = "A"
    B = "B"


class Response(BaseModel):
    result: list[int] = Field(..., description="3 random integers")
    value: Selector = Field(..., description="Choose a value randomly")


chatbot = LLMAgent[None, Response, None](
    name="chatbot",
    llm=OpenAILLM(
        model_name="google_ai_studio:gemini-2.0-flash",
        llm_settings=OpenAILLMSettings(use_structured_outputs=True),
        # response_format=Response,
    ),
)

# By default, response_format is set to the output type of the agent (Response)
# In some cases, you may want to set it to a different type, e.g. when using
# custom output parsing.

ctx = RunContext[None](print_messages=True)

In [10]:
out = await chatbot.run("start", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
start

<chatbot>[ASSISTANT]
{
  "result": [
    42,
    7,
    99
  ],
  "value": "B"
}

------------------------------------
I/O/(R)/(C) tokens: 21/33

<Response>
result=[42, 7, 99] value=<Selector.B: 'B'>


# Chat with images

In [11]:
chatbot = LLMAgent[None, str, None](
    name="chatbot", llm=OpenAILLM(model_name="openai:gpt-4.1")
)

ctx = RunContext[None](print_messages=True)

In [12]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
Where are you headed, stranger?

<chatbot>[ASSISTANT]
Just passing through the winding trails of conversation. Where do you reckon we’re headed next?

------------------------------------
I/O/(R)/(C) tokens: 14/18/0/0

<Response>
Just passing through the winding trails of conversation. Where do you reckon we’re headed next?


In [13]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
What did you just say, exactly?

<chatbot>[ASSISTANT]
I said, “Just passing through the winding trails of conversation. Where do you reckon we’re headed next?”

Care to take the lead?

------------------------------------
I/O/(R)/(C) tokens: 48/28/0/0

<Response>
I said, “Just passing through the winding trails of conversation. Where do you reckon we’re headed next?”

Care to take the lead?


In [14]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print_single_output(out)


<chatbot>[USER]
What's in this image?
<ENCODED_IMAGE>

<chatbot>[ASSISTANT]
The image contains a mathematical expression:

**7 * (5 + 15) / (2 * 5) - 3**

Would you like me to solve it?

------------------------------------
I/O/(R)/(C) tokens: 344/35/0/0

<Response>
The image contains a mathematical expression:

**7 * (5 + 15) / (2 * 5) - 3**

Would you like me to solve it?


In [15]:
out = await chatbot.run("Go on", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
Go on

<chatbot>[ASSISTANT]
Let’s solve the expression step by step:

**Expression:**  
7 * (5 + 15) / (2 * 5) - 3

1. Calculate inside the parentheses:  
   (5 + 15) = 20  
   (2 * 5) = 10

2. Substitute back:  
   7 * 20 / 10 - 3

3. Multiply:  
   7 * 20 = 140

4. Divide:  
   140 / 10 = 14

5. Subtract:  
   14 - 3 = **11**

**Final answer:**  
**11**

------------------------------------
I/O/(R)/(C) tokens: 389/135/0/0

<Response>
Let’s solve the expression step by step:

**Expression:**  
7 * (5 + 15) / (2 * 5) - 3

1. Calculate inside the parentheses:  
   (5 + 15) = 20  
   (2 * 5) = 10

2. Substitute back:  
   7 * 20 / 10 - 3

3. Multiply:  
   7 * 20 = 140

4. Divide:  
   140 / 10 = 14

5. Subtract:  
   14 - 3 = **11**

**Final answer:**  
**11**


In [16]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print_single_output(out)


<chatbot>[USER]
Try another one
https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG



<chatbot>[ASSISTANT]
Let's solve the given arithmetic expression with the provided values:

Given:  
a = 2  
b = 3  
c = 4

Expression:  
Z = a + b - (a * c)

Step-by-step:  
1. Multiply: a * c = 2 * 4 = 8  
2. Add: a + b = 2 + 3 = 5  
3. Subtract: 5 - 8 = **-3**

**Final Answer:**  
Z = **-3**

------------------------------------
I/O/(R)/(C) tokens: 960/108/0/0

<Response>
Let's solve the given arithmetic expression with the provided values:

Given:  
a = 2  
b = 3  
c = 4

Expression:  
Z = a + b - (a * c)

Step-by-step:  
1. Multiply: a * c = 2 * 4 = 8  
2. Add: a + b = 2 + 3 = 5  
3. Subtract: 5 - 8 = **-3**

**Final Answer:**  
Z = **-3**


In [17]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print_single_output(out)


<chatbot>[USER]
What was my first question, exactly?

<chatbot>[ASSISTANT]
Your first question was:

**"Where are you headed, stranger?"**

------------------------------------
I/O/(R)/(C) tokens: 1084/15/0/0

<Response>
Your first question was:

**"Where are you headed, stranger?"**


In [18]:
ctx.usage_tracker.total_usage

Usage(input_tokens=2839, output_tokens=339, reasoning_tokens=0, cached_tokens=0, cost=0.00839)

# Simple batching

In [19]:
sys_prompt = "You are a bad math student who always adds number {added_num} to the correct result of the operation."
usr_prompt = "What is the square of {num}?"


class SystemArgs(LLMPromptArgs):
    added_num: int


class UserArgs(LLMPromptArgs):
    num: int


student = LLMAgent[Any, str, None](
    name="student",
    llm=OpenAILLM(
        model_name="openai:gpt-4.1",
        # set the rate limit if desired
        rate_limiter_rpm=1000,
    ),
    sys_prompt=sys_prompt,
    sys_args_schema=SystemArgs,
    in_prompt=usr_prompt,
    usr_args_schema=UserArgs,
)

[OpenAILLM] Set rate limit to 1000 RPM


#### One system prompt -> many user arguments

In [20]:
run_args = RunArgs(
    sys=SystemArgs(added_num=1),
    usr=[UserArgs(num=i) for i in range(1, 10)],
)

ctx = RunContext[None](run_args={"student": run_args}, print_messages=True)

In [21]:
out = await student.run(ctx=ctx)

print()
print(*[p for p in out.payloads], sep="\n")


<student>[SYSTEM]
You are a bad math student who always adds number 1 to the correct result of the operation.
Message batch size is 9, current batch size is 1: duplicating the conversation to match the message batch size

<student>[USER]
What is the square of 1?

<student>[USER]
What is the square of 2?

<student>[USER]
What is the square of 3?

<student>[USER]
What is the square of 4?

<student>[USER]
What is the square of 5?

<student>[USER]
What is the square of 6?

<student>[USER]
What is the square of 7?

<student>[USER]
What is the square of 8?

<student>[USER]
What is the square of 9?

<student>[ASSISTANT]
The square of 1 is 2.

------------------------------------
I/O/(R)/(C) tokens: 39/9/0/0

<student>[ASSISTANT]
The square of 2 is 5.

------------------------------------
I/O/(R)/(C) tokens: 39/9/0/0

<student>[ASSISTANT]
Okay, so the square of 3 is 9. But since I always add 1, my answer would be 10.

------------------------------------
I/O/(R)/(C) tokens: 39/27/0/0

<studen

#### Many back to one

Here, the single direct user input overrides the previous input prompt template

In [22]:
out = await student.run(
    "Who are you, dear stranger? What was your last chore?", ctx=ctx
)

print()
print(*[p for p in out.payloads], sep="\n")

Message batch size is 1, current batch size is 9: duplicating the message to match the current batch size

<student>[USER]
Who are you, dear stranger? What was your last chore?

<student>[ASSISTANT]
I’m just your not-so-great math buddy! My last chore was “helping” someone with math—though I sorta messed up by always adding 1 to every answer. Oops! Want me to tackle another math problem (in my special way)?

------------------------------------
I/O/(R)/(C) tokens: 69/53/0/0

<student>[ASSISTANT]
I’m your friendly (but not-so-good-at-math) assistant! As for my last chore, I just finished helping someone with a math question—though I always manage to be off by 1 in my answers.

------------------------------------
I/O/(R)/(C) tokens: 69/45/0/0

<student>[ASSISTANT]
Haha, I’m just a bad math student who always answers math questions wrong by adding 1 to the real answer! As for my last chore… let's say I swept the floor, but I counted 1 extra corner when cleaning. Classic me!

------------

In [23]:
ctx.usage_tracker.total_usage

Usage(input_tokens=990, output_tokens=550, reasoning_tokens=0, cached_tokens=0, cost=0.006379999999999999)

# ReAct agent loop 

In [24]:
sys_prompt_react = """
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.
"""

In [25]:
# Tool input must be a Pydantic model to infer the JSON schema used by the LLM APIs
class TeacherQuestion(BaseModel):
    question: str


StudentReply = str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    reply: str
        The student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description

    async def run(
        self, inp: TeacherQuestion, ctx: RunContext[Any] | None = None
    ) -> StudentReply:
        return input(inp.question)

In [26]:
Problem = str


teacher = LLMAgent[None, Problem, None](
    name="teacher",
    llm=OpenAILLM(
        model_name="openai:gpt-4.1", llm_settings=OpenAILLMSettings(temperature=0.5)
    ),
    tools=[AskStudentTool()],
    max_turns=20,
    react_mode=True,
    sys_prompt=sys_prompt_react,
    reset_memory_on_run=True,
)


@teacher.exit_tool_call_loop
def exit_tool_call_loop(conversation: Messages, ctx, **kwargs: Any) -> bool:
    return r"<PROBLEM>" in str(conversation[-1].content)


@teacher.parse_output
def parse_output(conversation: Messages, ctx, **kwargs: Any) -> Problem:
    message = str(conversation[-1].content)
    matches = re.findall(r"<PROBLEM>(.*?)</PROBLEM>", message, re.DOTALL)

    return matches[0]

In [27]:
ctx = RunContext[None](print_messages=True)

In [28]:
out = await teacher.run("start", ctx=ctx)
print_single_output(out)


<teacher>[SYSTEM]
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Use the provided tool to ask questions.
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.

<teacher>[USER]
start

<teacher>[ASSISTANT]
Great! Before I suggest an exciting statistics problem, I'd like to get to know you a bit better so I can tailor the problem to your interests and background.

First, I'm thinking it would help to know your current level of education or experience with statistics. This way, I can make sure the problem is appropriate for your skill level.

Let me ask you about that now.

--

# Sequential workflow 

In [24]:
# Input arguments are passed to the agent dynamically (e.g. by other agents)
class AddInputArgs(BaseModel):
    a: int = Field(..., description="First number to add.")


# User arguments are passed to the agent statically via run_args in RunContextWrapper
class AddUserArgs(LLMPromptArgs):
    b: int


class AddResponse(BaseModel):
    a_plus_b: int


# The input prompt template is used to combine the user and received arguments
# and format them for the LLM.
add_in_prompt = "Add {a} and {b}. Your only output is the resulting number."


add_agent = LLMAgent[AddInputArgs, AddResponse, None](
    name="add_agent",
    llm=OpenAILLM(model_name="openai:gpt-4.1"),
    usr_args_schema=AddUserArgs,
    in_prompt=add_in_prompt,
    # Reset message history to system prompt (if provided) before each run
    reset_memory_on_run=True,
)


@add_agent.parse_output
def parse_output(conversation: Messages, **kwargs) -> AddResponse:
    return AddResponse(a_plus_b=int(str(conversation[-1].content)))

In [25]:
class MultiplyUserArgs(LLMPromptArgs):
    c: int


class MultiplyResponse(BaseModel):
    c_a_plus_b: int


multiply_in_prompt = (
    "Multiply {a_plus_b} by {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, None](
    name="multiply_agent",
    llm=OpenAILLM(model_name="openai:gpt-4.1"),
    usr_args_schema=MultiplyUserArgs,
    in_prompt=multiply_in_prompt,
    reset_memory_on_run=True,
)


@multiply_agent.parse_output
def parse_output(conversation: Messages, **kwargs) -> MultiplyResponse:
    return MultiplyResponse(c_a_plus_b=int(str(conversation[-1].content)))

In [26]:
seq_agent = SequentialWorkflow[AddInputArgs, MultiplyResponse, None](
    name="seq_agent",
    subprocessors=[add_agent, multiply_agent],
)

In [27]:
add_run_args = RunArgs(usr=AddUserArgs(b=3))
multiply_run_args = RunArgs(usr=MultiplyUserArgs(c=6))

ctx = RunContext[None](
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=True,
)

In [28]:
# Can pass batched arguments
out = await seq_agent.run(in_args=[AddInputArgs(a=2)], ctx=ctx)
print_single_output(out)


<add_agent>[USER]
Add 2 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
5

------------------------------------
I/O/(R)/(C) tokens: 22/1/0/0

<multiply_agent>[USER]
Multiply 5 by 6. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
30

------------------------------------
I/O/(R)/(C) tokens: 22/1/0/0

<Response>
c_a_plus_b=30


# Agents as tools

When agents are used as tools, their `in_args` become the tool inputs.

This is how one can implement a manager + helpers architecture.

In [30]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5."
    ),
)

The JSON schema of `in_args` is preserved:

In [31]:
seq_tool.in_schema.model_json_schema()

{'properties': {'a': {'description': 'First number to add.',
   'title': 'A',
   'type': 'integer'}},
 'required': ['a'],
 'title': 'AddInputArgs',
 'type': 'object'}

In [32]:
await seq_tool(a=15, ctx=ctx)


<add_agent>[USER]
Add 15 and 3. Your only output is the resulting number.

<add_agent>[ASSISTANT]
18

------------------------------------
I/O/(R)/(C) tokens: 22/1/0/0

<multiply_agent>[USER]
Multiply 18 by 6. Your only output is the resulting number.

<multiply_agent>[ASSISTANT]
108

------------------------------------
I/O/(R)/(C) tokens: 22/1/0/0


MultiplyResponse(c_a_plus_b=108)

# Teacher / students

A more advanced example of multi-agent debate, where agents communicate using the actor model.

In [40]:
def extract_recipients(message: str) -> list[str]:
    match = re.search(r"\[(.*?)\]", message)
    if match:
        # Extract the contents inside square brackets
        content = match.group(1)
        # Extract each student name within angle brackets
        return re.findall(r"<(.*?)>", content)  # Output: ['Alice', 'Bob', 'Charlie']

    return []

Communication schemas

In [41]:
# Teacher can choose which students to send the message to
# We need to inherit from DynCommPayload to use dynamic communication
class TeacherExplanation(DynCommPayload):
    explanation: str


# Students can only ask questions to the teacher
class StudentQuestion(BaseModel):
    question: str

In [42]:
pool = PacketPool[Any]()

#### Teacher

In [43]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    name="teacher",
    llm=OpenAILLM(model_name="openai:gpt-4o"),
    sys_prompt=teacher_sys_prompt,
    packet_pool=pool,
    # all available recipients to choose from:
    recipients=["student1", "student2"],
)


@teacher.parse_output
def parse_teacher_output(conversation: Messages, **kwargs) -> TeacherExplanation:
    message = str(conversation[-1].content)
    # Quick and dirty regexes to extract the recipients and explanation
    recipients = extract_recipients(message)
    explanation = message.split("[")[0].strip()

    # `selected_recipient_ids` is a required field for `DynCommPayload`
    return TeacherExplanation(explanation=explanation, selected_recipients=recipients)


@teacher.exit_communication
def teacher_exit_condition(out_packet: Packet[TeacherExplanation], ctx) -> bool:
    # Finish the conversation if the teacher says "Goodbye, students!"
    message = out_packet.payloads[0].explanation

    return "Goodbye, students!" in message

#### Students

In [44]:
student_sys_prompts = [
    """
You are a 4-year old child trying to make sense of physics. 
Your name is <student1>.
Talk to the teacher to understand the topic.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 4-year old child.
You talk to the teacher only.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    return LLMAgent[TeacherExplanation, StudentQuestion, None](
        name=name,
        llm=OpenAILLM(model_name="openai:gpt-4o"),
        sys_prompt=sys_prompt,
        packet_pool=pool,
        recipients=["teacher"],
    )


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])


@student1.parse_output
def parse_student1_output(conversation: Messages, **kwargs) -> StudentQuestion:
    return StudentQuestion(question="<student1>: " + str(conversation[-1].content))


@student2.parse_output
def parse_student2_output(conversation: Messages, **kwargs) -> StudentQuestion:
    return StudentQuestion(question="<student2>: " + str(conversation[-1].content))

Specify shared context 

In [45]:
ctx = RunContext[None](print_messages=True)
ctx.printer.color_by = "agent"

Run and wait until completion

In [ ]:
await teacher.start_listening(ctx=ctx)
await student1.start_listening(ctx=ctx)
await student2.start_listening(ctx=ctx)

# Teacher starts the conversation by posting a message to the pool
await teacher.run_and_post(ctx=ctx)